In [269]:
import torch
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")


train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')


Using device: mps


**Train**

42000*785
(label / pixel 0~783)

**28*28 pixel**

In [270]:
print(train.shape) 
print(test.shape)
#print(train.head())
print(train.head())

(42000, 785)
(28000, 784)
   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0         0         0         0   
3       0  ...         0         0         0         0         0         0   
4       0  ...         0         0         0         0         0         0   

   pixel780  pixel781  pixel782  pixel783  
0   

In [271]:
x_train=train.values[:,1:]
y_train=train.values[:,:1]
x_test=test.values
x_train.shape


(42000, 784)

In [272]:
class NN(nn.Module):
    def __init__(self,num_classes):
        super(NN,self).__init__()
        self.layer1=nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,stride=1,padding=1),
            #nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2), #14*14
        )
        self.layer2=nn.Sequential(
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=1),
            #nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2) #7*7
        )
        self.fcLayer=nn.Sequential(
            #nn.Dropout(p=0.5),
            nn.Linear(7*7*64,512),
            #nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            #nn.Dropout(p=0.5),
            nn.Linear(512,128),
            #nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            #nn.Dropout(p=0.5),
            nn.Linear(128,num_classes)
        )
    def forward(self, x):
        x=self.layer1(x)
        x=self.layer2(x)
        x=x.reshape(x.shape[0],-1)
        x=self.fcLayer(x)
        return x


In [273]:

x_train=x_train.reshape(x_train.shape[0],1,28,28)
x_test=x_test.reshape(x_test.shape[0],1,28,28)
y_train=y_train.squeeze()
print(type(y_train), y_train.shape)
#7*7*64

<class 'numpy.ndarray'> (42000,)


Epoch 1/10, Prediction : 39584/42000, Cost: 0.18144252613240056

Epoch 2/10, Prediction : 41307/42000, Cost: 0.05401082083658271

Epoch 3/10, Prediction : 41526/42000, Cost: 0.03682552347456416

Epoch 4/10, Prediction : 41655/42000, Cost: 0.026996741560596827

Epoch 5/10, Prediction : 41741/42000, Cost: 0.019561439031299546


**With BatchNorm**

Epoch 1/10, Prediction : 40535/42000, Cost: 0.11206443339332361

Epoch 2/10, Prediction : 41432/42000, Cost: 0.04623741868974563

Epoch 3/10, Prediction : 41545/42000, Cost: 0.03753566086195947

Epoch 4/10, Prediction : 41665/42000, Cost: 0.026764186040688878

Epoch 5/10, Prediction : 41740/42000, Cost: 0.020635666684884892




In [274]:
learning_rate=1E-3
model = NN(10).to(device)

optim=torch.optim.Adam(model.parameters(),lr=learning_rate)
loss_fn=nn.CrossEntropyLoss()
epoch=20
batch_size=32

#transform=transforms.Normalize(0.5,0.5)
#x_train=transform(x_train)
x_train=x_train/256.0
x_test=x_test/256.0
x_train=torch.FloatTensor(x_train)
x_test=torch.FloatTensor(x_test)
y_train=torch.FloatTensor(y_train)

dataset=TensorDataset(x_train,y_train)
dataloader=DataLoader(dataset,batch_size=batch_size,shuffle=True)

train_loss_min=1E+9
for epoch_cnt in range(epoch):
    num_right=0
    train_loss=0
    for index, (data,ans) in enumerate(dataloader):
        data, ans = data.to(device), ans.to(device)
        optim.zero_grad()
        prediction=model(data)
        loss=loss_fn(prediction,ans.long())
        loss.backward()
        optim.step()
        labels=torch.argmax(prediction,dim=1)
        num_right+=torch.sum(labels==ans)
        #print('{} / {}'.format(torch.sum(labels==ans),batch_size))
        train_loss+=loss.item()*len(data)
    train_loss=train_loss/len(x_train)
    print('Epoch {}/{}, Prediction : {}/{}, Cost: {}'.format(epoch_cnt+1,epoch,num_right,len(x_train),train_loss))
    if(train_loss<train_loss_min):
        print('=*=*=*= Loss decreased ({:6f} ===> {:6f}). Saving the model! =*=*=*='.format(train_loss_min, train_loss))
        torch.save(model.state_dict(), 'model.pt')
        train_loss_min = train_loss


Epoch 1/20, Prediction : 39357/42000, Cost: 0.20177850348626575
=*=*=*= Loss decreased (1000000000.000000 ===> 0.201779). Saving the model! =*=*=*=
Epoch 2/20, Prediction : 41060/42000, Cost: 0.07183581211945662
=*=*=*= Loss decreased (0.201779 ===> 0.071836). Saving the model! =*=*=*=
Epoch 3/20, Prediction : 41257/42000, Cost: 0.05439175357124103
=*=*=*= Loss decreased (0.071836 ===> 0.054392). Saving the model! =*=*=*=
Epoch 4/20, Prediction : 41412/42000, Cost: 0.04559983406891115
=*=*=*= Loss decreased (0.054392 ===> 0.045600). Saving the model! =*=*=*=
Epoch 5/20, Prediction : 41471/42000, Cost: 0.03913248048431151
=*=*=*= Loss decreased (0.045600 ===> 0.039132). Saving the model! =*=*=*=
Epoch 6/20, Prediction : 41573/42000, Cost: 0.03241334628515012
=*=*=*= Loss decreased (0.039132 ===> 0.032413). Saving the model! =*=*=*=
Epoch 7/20, Prediction : 41625/42000, Cost: 0.02818414183080867
=*=*=*= Loss decreased (0.032413 ===> 0.028184). Saving the model! =*=*=*=
Epoch 8/20, Predic

In [275]:
#model = torch.load("./model.pt")
dataset=TensorDataset(x_test)
dataloader=DataLoader(dataset,batch_size=batch_size,shuffle=False)
test_preds=torch.LongTensor()
for index, (data,) in enumerate(dataloader):
    with torch.no_grad():
        data=data.to(device)
        result=model(data)
        output=torch.argmax(result,dim=1)
        print(output)
        test_preds = torch.cat((test_preds, output.cpu()), dim=0)

tensor([2, 0, 9, 0, 3, 7, 0, 3, 0, 3, 5, 7, 4, 0, 4, 3, 3, 1, 9, 0, 9, 1, 1, 5,
        7, 4, 2, 7, 4, 7, 7, 5], device='mps:0')
tensor([4, 2, 6, 2, 5, 5, 1, 6, 7, 7, 4, 9, 8, 7, 8, 2, 6, 7, 6, 8, 8, 3, 8, 2,
        1, 2, 2, 0, 4, 1, 7, 0], device='mps:0')
tensor([0, 0, 1, 9, 0, 1, 6, 5, 8, 8, 2, 8, 9, 9, 2, 3, 5, 4, 1, 8, 9, 2, 4, 3,
        6, 7, 2, 0, 6, 6, 1, 4], device='mps:0')
tensor([3, 9, 7, 4, 0, 9, 2, 0, 7, 3, 0, 5, 0, 8, 0, 0, 4, 7, 1, 7, 1, 1, 3, 3,
        3, 7, 2, 8, 6, 3, 8, 7], device='mps:0')
tensor([7, 4, 3, 5, 6, 0, 0, 0, 3, 1, 3, 6, 4, 3, 4, 5, 5, 8, 7, 7, 2, 8, 4, 3,
        5, 6, 5, 3, 7, 5, 7, 8], device='mps:0')
tensor([3, 0, 4, 5, 1, 2, 7, 6, 3, 0, 2, 7, 8, 6, 1, 3, 7, 4, 1, 2, 4, 8, 5, 2,
        4, 9, 2, 1, 6, 0, 6, 1], device='mps:0')
tensor([4, 9, 6, 0, 9, 7, 6, 9, 1, 9, 0, 9, 9, 0, 8, 4, 6, 2, 0, 9, 3, 6, 3, 2,
        1, 6, 3, 4, 2, 3, 1, 2], device='mps:0')
tensor([2, 0, 4, 6, 1, 0, 0, 4, 9, 1, 7, 3, 2, 3, 8, 6, 8, 6, 2, 8, 5, 5, 9, 8,
        3, 5, 9, 

In [276]:
#answer = all_labels.numpy()
submission=pd.read_csv("../data/sample_submission.csv")
submission['Label']=test_preds.numpy().squeeze()
submission.head()
submission.to_csv('submission.csv', index=False)

In [277]:
tmp=x_test[2][0].data
for i in range(28):
    for j in range(28):
        if(tmp[i][j].item()>0): 
            tmp[i][j]=1
        print(int(tmp[i][j].item()),end="")
    print("")

0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000111111110000000
0000000000001111111110000000
0000000000011110000111000000
0000000000111000000111000000
0000000000111000001110000000
0000000001110000011100000000
0000000001100001111100000000
0000000001101111111000000000
0000000001111111110000000000
0000000000111111110000000000
0000000000000011100000000000
0000000000000111100000000000
0000000000001111000000000000
0000000000001110000000000000
0000000000011100000000000000
0000000000111100000110000000
0000000000111000000111100000
0000000001110000000111100000
0000000011100000000000000000
0000000011100000000000000000
0000000000000000000000000000
0000000000000000000000000000


**출처**

https://www.kaggle.com/code/franklemuchahary/mnist-digit-recognition-using-pytorch